In [ ]:
import sys
sys.path.append('../python/')
sys.path.append('../data/')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader

import data_manual as data_utils

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
print (torch.__version__)
print(torch.cuda.current_device())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu:0')

In [ ]:
from vgg import VGG16

In [ ]:
from torch.hub import load_state_dict_from_url
vgg16 = VGG16(in_channels=3, output_dim=1000).to(device)
# before = vgg_features.state_dict()['fc7.weight']
state_dict = load_state_dict_from_url(url=
'https://download.pytorch.org/models/vgg16-397923af.pth')

vgg16.load_state_dict(state_dict,strict=False)

In [ ]:
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from os import path

IMAGE_SIZE = 64

In [ ]:
def get_vgg_features(vgg16, loader, num_points, block_idx):
    features = np.zeros([num_points, 4096], dtype=np.float32)
    labels = []

    idx = 0
    prev_idx = 0
    for batch_x, batch_y in tqdm(iter(loader)):
        batch_x = (batch_x + 1) / 2 #Normalize to 0, 1
        if idx == 0:
            print('Input range: [{}, {}]'.format(batch_x.min().item(), batch_x.max().item()))
        #check the batch data, make sure it's in the appropriate range
        assert batch_x.min().item() >= 0
        assert batch_x.max().item() <= 1
        idx = len(batch_x) + prev_idx
        features[prev_idx: idx] = vgg16.get_fc7(batch_x.to(device), idx=block_idx).cpu().numpy()
        prev_idx = idx
        labels += batch_y.detach().cpu().numpy().tolist()
    return features, labels

def get_vgg_data(dataroot, seed, num_train, num_db, num_query, block_idx):    
    train_loader =  data_utils.get_dataloader(dataset, IMAGE_SIZE, 100,
                       dataroot=dataroot, workers=0, data_transforms=None, type='train', shuffle=False, clear_cache=True)
    db_loader =  data_utils.get_dataloader(dataset, IMAGE_SIZE, 100,
                       dataroot=dataroot, workers=0, data_transforms=None, type='db', shuffle=False, clear_cache=True)
    query_loader =  data_utils.get_dataloader(dataset, IMAGE_SIZE, 100,
                       dataroot=dataroot, workers=0, data_transforms=None, type='query', shuffle=False, clear_cache=True)
    
    with torch.no_grad():
        train_features, train_labels = get_vgg_features(vgg16, train_loader, num_train, block_idx)
        print('train done: data range [{},{}]'.format(train_features.min(), train_features.max()))

        query_features, query_labels = get_vgg_features(vgg16, query_loader, num_query, block_idx)
        print('query done: data range [{},{}]'.format(train_features.min(), train_features.max()))

        db_features, db_labels = get_vgg_features(vgg16, db_loader, num_db, block_idx)
        print('db done: data range [{},{}]'.format(db_features.min(), db_features.max()))
    return train_features, train_labels, query_features, query_labels, db_features, db_labels

## MNIST

Using the created 3-channel mnist dataset in `create_datasets.ipynb`, we can generate the VGG features.

In [ ]:
dataset = 'mnist-3c'
block_idx = 2
for (seed, num_train, num_db, num_query) in [
        (9, 60000, 60000, 10000),
        (19, 60000, 60000, 10000),
        (29, 60000, 60000, 10000),
        (39, 60000, 60000, 10000),
        (49, 60000, 60000, 10000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=block_idx
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'db')), x = db_features, y=db_labels)

In [ ]:
dataset = 'mnist-3c'
block_idx = 1
for (seed, num_train, num_db, num_query) in [
        (9, 60000, 60000, 10000),
        (19, 60000, 60000, 10000),
        (29, 60000, 60000, 10000),
        (39, 60000, 60000, 10000),
        (49, 60000, 60000, 10000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=block_idx
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'db')), x = db_features, y=db_labels)

In [ ]:
dataset = 'mnist-3c'
block_idx = 1
for (seed, num_train, num_db, num_query) in [
        (109, 5000, 55000, 10000),
        (119, 5000, 55000, 10000),
        (129, 5000, 55000, 10000),
        (139, 5000, 55000, 10000),
        (149, 5000, 55000, 10000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=block_idx
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'db')), x = db_features, y=db_labels)

In [ ]:
dataset = 'mnist-3c'
block_idx = 2
for (seed, num_train, num_db, num_query) in [
        (109, 5000, 55000, 10000),
        (119, 5000, 55000, 10000),
        (129, 5000, 55000, 10000),
        (139, 5000, 55000, 10000),
        (149, 5000, 55000, 10000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=block_idx
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'db')), x = db_features, y=db_labels)

## CIFAR-10

In [ ]:
dataset = 'cifar10'

for (seed, num_train, num_db, num_query) in [
        (9, 50000, 50000, 10000),
        (19, 50000, 50000, 10000),
        (29, 50000, 50000, 10000),
        (39, 50000, 50000, 10000),
        (49, 50000, 50000, 10000),
    ]:    
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=2
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_2_{}_manual_{}.npz'.format(dataset, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_2_{}_manual_{}.npz'.format(dataset, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_2_{}_manual_{}.npz'.format(dataset, 4096, 'db')), x = db_features, y=db_labels)

In [ ]:
dataset = 'cifar10'
block_idx = 1

for (seed, num_train, num_db, num_query) in [
        (9, 50000, 50000, 10000),
        (19, 50000, 50000, 10000),
        (29, 50000, 50000, 10000),
        (39, 50000, 50000, 10000),
        (49, 50000, 50000, 10000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=block_idx
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(dataset, block_idx, 4096, 'db')), x = db_features, y=db_labels)

## FLICKR-25K

In [ ]:
dataset = 'flickr25k'

for (seed, num_train, num_db, num_query) in [
        (9, 20000, 20000, 5000),
        (19, 20000, 50000, 5000),
        (29, 20000, 20000, 5000),
        (39, 20000, 20000, 5000),
        (49, 20000, 20000, 5000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=2
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_2_{}_manual_{}.npz'.format(dataset, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_2_{}_manual_{}.npz'.format(dataset, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_2_{}_manual_{}.npz'.format(dataset, 4096, 'db')), x = db_features, y=db_labels)

In [ ]:
dataset = 'flickr25k'
block_idx = 1

for (seed, num_train, num_db, num_query) in [
        (9, 20000, 20000, 5000),
        (19, 20000, 50000, 5000),
        (29, 20000, 20000, 5000),
        (39, 20000, 20000, 5000),
        (49, 20000, 20000, 5000),
    ]:
    dataroot = '../data/{}_isize{}_seed{}'.format(dataset, IMAGE_SIZE, seed)
    train_features, train_labels, query_features, query_labels, db_features, db_labels = get_vgg_data(
        dataroot, seed, num_train, num_db, num_query, block_idx=2
    )
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(block_idx, dataset, 4096, 'query')), x = query_features, y=query_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(block_idx, dataset, 4096, 'train')), x = train_features, y=train_labels)
    np.savez_compressed(path.join(dataroot, '{}_fc7_{}_{}_manual_{}.npz'.format(block_idx, dataset, 4096, 'db')), x = db_features, y=db_labels)

# END